<a href="https://colab.research.google.com/github/tirals88/Deep-Learning-from-Scratch/blob/main/1%EA%B6%8C_5%EC%9E%A5_(1)_%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#드라이브연동
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
cd '/content/drive/My Drive/DLscratch/deep-learning-from-scratch/ch05'

# 오차역전파법

앞 장에서는 신경망 학습에 대해서 설명을 하였다. 그 때는 가중치 매개변수의 손실함수의 기울기를 수치 미분을 사용하여 구하였다.
수치미분은 구현이 쉽고 단순하지만 오래걸린다는 단점이 있었다. 이번에는 효율적으로 계산하는 '오차역전파법'을 배워볼 것이다.

먼저 시작적 ( 계산 그래프를 통하여 )으로 이해를 해보자.

오차역전파법을 계산 그래프로 설명한다는 생각은 **Andrej Karpathy**의 블로그, **Fei-Fei Li** 교수가 진행한 스탠퍼드 대학교의 딥러닝 수업 CS231n을 참고하였다고한다.
물론 내가 정리한 모든 내용들 또한 **밑바닥부터 시작하는 딥러닝(Deep Learning from Scratch)**의 내용들이다.

## 계산그래프

그림을 통한 이해는 책을 통해 복습
https://github.com/WegraLee/deep-learning-from-scratch/tree/master/ch05



계산 그래프의 특징은 '국소적 계산'을 전파함으로써 최종 결과를 얻는다는 점에 있다.

국소적 계산은 결국 전체에서 어떤 일이 벌어지든 상관없이 독립적으로 자신과 관계된 정보만으로 결과를 출력할 수 있다는 것이다.

👉 각 노드는 자신과 관련된 계산 외에는 아무것도 신경 쓸 게 없다.

국소적 계산의 장점
1. 전체가 아무리 복잡하더라도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화 할 수 있다.
2. 중간 계산 결과를 모두 보관할 수 있다.
3. **역전파를 통해 '미분'을 효율적으로 계산할 수 있다.**



## 연쇄법칙

합성함수
- 여러 함수로 굿어된 함수이다.
$z = (x + y)^2$, 이 식은 아래의 두 개의 식으로 구성된 것이다.
$$z = t^2$$
$$t = x + y$$

- 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타내진다.
$$\frac{\partial z}{\partial x} = \frac{∂z}{∂t}\frac{∂t}{∂x}$$

위 식에서 $∂t$는 서로 지울 수 있다.

$$\frac{∂z}{∂t} = 2t$$

$$\frac{∂t}{∂x} = 1$$

$$\frac{\partial z}{\partial x} = \frac{∂z}{∂t}\frac{∂t}{∂x} = 2t ⋅1 = 2(x+y)$$

- 국소적 미분의 전달

$→ \frac{\partial z}{\partial z}$ 

$→ \frac{∂z}{∂z}\frac{∂z}{∂t}$ 

$→ \frac{\partial z}{\partial z}\frac{∂z}{∂t}\frac{∂t}{∂x}$

이는 결국 $x$에 대한 $z$의 미분이 된다.

## 덧셈 노드의 역전파

$z = x + y$에서 $\frac{∂z}{∂x} = 1$, $\frac{∂z}{∂y} = 1$이 된다.

그러므로 덧셈 노드에서 역전파는 상류에서 전해진 미분 ( $\frac{∂L}{∂z}$에 1을 곱하여 하류로 흘려보내는 것이다. 즉 1을 곱하기만 할 분이므로 입력된 값을 그대로 다음 노드로 흘려보내게 된다.

덧셈 노드 역전파는 입력 신호를 다음 노드로 출력할 뿐이므로 그대로 다음 노드로 전달한다.

## 곱셈 노드의 역전파

$z = x ⋅ y$에서 $\frac{∂z}{∂x} = y$, $\frac{∂z}{∂y} = x$가 된다.

곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값'을 곱해서 하류로 보낸다.

**덧셈의 역전파는 상류의 값을 그대로 흘려보내기 대문에 순방향 입력 신호의 값은 필요하지 않았지만,**

**곱셈의 역전파는 순방향 입력 신호의 값이 필요하다. 그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장해둔다.**